#RNN Word predictor

In [32]:
import pandas as pd
df = pd.read_csv('100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


#Make the unique words in to number

#1tokenize

In [33]:
# tokenize means separating every words
def tokenize(text):
  text = text.lower() #convert everything to lowercasse
  text = text.replace('?','')#replace ? with nothing
  text = text.replace("'","")#replace ""  with nothing
  return text.split()

In [34]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

#2.Vocabulary form

In [35]:
#dictionary to hold words
vocab = {'<UNK>':0} #unknown words ->UNK meaning

In [36]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [37]:
#total vocabulary and vocab size
print(f'Vocab size before building:{len(vocab)}')
print("\n ")
print(vocab)

Vocab size before building:1

 
{'<UNK>': 0}


In [38]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

#3.Convert words to numerical indices

In [39]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [40]:
text_to_indices("What is Engineering?", vocab)

[1, 2, 0]

#Dataset Dataloader

In [41]:
import torch
from torch.utils.data import Dataset, DataLoader

In [42]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [43]:
dataset = QADataset(df, vocab)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [56]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

#full question and answers

In [44]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 10, 140,   3, 141, 142,  12, 143,  83,   3, 144]]) tensor([145])
tensor([[10, 75, 76]]) tensor([77])
tensor([[  1,   2,   3,   4,   5, 206]]) tensor([207])
tensor([[ 42, 250, 251, 118, 252, 253]]) tensor([254])
tensor([[10, 96,  3, 97]]) tensor([98])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([16])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([91])
tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([298])
tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([36])
tensor([[  1,   2,   3,   4,   5, 109]]) tensor([317])
tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([85])
tensor([[  1,   2,   3,   4,   5, 279]]) tensor([280])
tensor([[ 42, 290, 291, 118, 292, 158, 293, 294]]) tensor([295])
tensor([[ 10,  75, 208]]) tensor([209])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([246])
tensor([[  1,   2,   3, 146,  86,  19, 192, 193]]) tensor([194])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) 

#model

In [45]:
import torch.nn as nn

class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [46]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [47]:
learning_rate = 0.001
epochs = 20

model = SimpleRNN(len(vocab))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#Training

In [48]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 524.616419
Epoch: 2, Loss: 455.184950
Epoch: 3, Loss: 375.979933
Epoch: 4, Loss: 312.849213
Epoch: 5, Loss: 260.914947
Epoch: 6, Loss: 213.497490
Epoch: 7, Loss: 170.638049
Epoch: 8, Loss: 133.192959
Epoch: 9, Loss: 102.126788
Epoch: 10, Loss: 77.717646
Epoch: 11, Loss: 60.026411
Epoch: 12, Loss: 47.133650
Epoch: 13, Loss: 37.084747
Epoch: 14, Loss: 30.146931
Epoch: 15, Loss: 24.985163
Epoch: 16, Loss: 20.682679
Epoch: 17, Loss: 17.633343
Epoch: 18, Loss: 15.032403
Epoch: 19, Loss: 12.975315
Epoch: 20, Loss: 11.233914


In [49]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [53]:
predict(model, "What is the Capital of India?")

delhi


In [55]:
list(vocab.keys())[5]

'of'